# Overview
    
This Project is based on the dataset available at Kaggle which is composed ofabout 20K tweets to train sentiment predictors.
This Notebook is going to train a model that predict the sentiment of a text/tweet, This Notebook will guide you through the process of training the model from data prepration to data cleaning & training some Deep Learing models to predict the       result.

In [1]:
import pandas as pd

# Data Importing

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


# Data Exploration

In [4]:
df.head(20)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
5,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,http://www.dothebouncy.com/smf - some shameles...,neutral
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive
7,50e14c0bb8,Soooo high,Soooo high,neutral
8,e050245fbd,Both of you,Both of you,neutral
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive


In [5]:
# Length of the dataset
len(df)

27481

In [6]:
#Is there any other different value than neutral , positive & negative?
df['sentiment'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [7]:
#Data is biased ?
df.groupby('sentiment').nunique()

,textID,text,selected_text
sentiment,,,
negative,7781,7781,5861
neutral,11118,11117,11111
positive,8582,8582,5537


Data is little bit biased

#  Data Cleaning

In [8]:
#The columns we are going to use are sentiment & selected_text
train = df[['selected_text' , 'sentiment']]
train.head(6)

,selected_text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD,negative
2,bullying me,negative
3,leave me alone,negative
4,"Sons of ****,",negative
5,http://www.dothebouncy.com/smf - some shameles...,neutral


In [9]:
#Is there any null values?
train["selected_text"].isnull().sum()

1

In [10]:
#Filling the null value
train["selected_text"].fillna("No Content" , inplace= True)

C:\Users\Acer\AppData\Local\Temp\ipykernel_16952\3312253806.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["selected_text"].fillna("No Content" , inplace= True)


The next step about data cleaning will be:

1.Removing URLs
2.Tokenize text
3.Removing Emails
4.Removing new line characters
5.Removing distracting single quotes
6.Removing all punctuation signs
7.Lowercase all text
8.Detokenize text
9.Convert list of texts to numpy array

In [11]:
import numpy as np
import re
import gensim
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [12]:
def depure_data(data):
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www.\S+')
    data = url_pattern.sub(r'' , data)
    
    #Removing Emails
    data = re.sub('\S*@\S*\s?' , '', data)
    
    #Removing new line characters
    data = re.sub('s+' , ' ', data)
    # Removing distracting single quotes
    data = re.sub("\'" , "" , data)
    
    return data

In [13]:
temp = []
#Splitting pd series to list
data_to_list = train['selected_text'].values.tolist()
for i in range(len(data_to_list)):
    temp.append(depure_data(data_to_list[i]))
list(temp[:5])

['I`d have re ponded, if I were going',
 'Sooo SAD',
 'bullying me',
 'leave me alone',
 'Son  of ****,']

In [14]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence) , deacc = True))  
        #deacc ture will remove punctuaions
        
data_words = list(sent_to_words(temp))

print(data_words[:20])


[['have', 're', 'ponded', 'if', 'were', 'going'], ['sooo', 'sad'], ['bullying', 'me'], ['leave', 'me', 'alone'], ['son', 'of'], ['ome', 'hamele', 'plugging', 'for', 'the', 'be', 'ranger', 'forum', 'on', 'earth'], ['fun'], ['soooo', 'high'], ['both', 'of', 'you'], ['wow', 'ju', 'became', 'cooler'], ['much', 'love', 'to', 'be', 'hopeful', 'reckon', 'the', 'chance', 'are', 'minimal', 'never', 'gonna', 'get', 'my', 'cake', 'and', 'tuff'], ['like'], ['dangerou', 'ly'], ['lo'], ['te', 'te', 'from', 'the', 'lg', 'env'], ['uh', 'oh', 'am', 'unburned'], ['igh'], ['ick'], ['onna'], ['he', 'ju', 'not', 'that', 'into', 'you']]


In [15]:
len(data_words)

27481

In [16]:
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

In [17]:
data = []
for i in range(len(data_words)):
    data.append(detokenize(data_words[i]))
    
print(data[:10])

['have re ponded if were going', 'sooo sad', 'bullying me', 'leave me alone', 'son of', 'ome hamele plugging for the be ranger forum on earth', 'fun', 'soooo high', 'both of you', 'wow ju became cooler']


In [18]:
data = np.array(data)

In [19]:
data

array(['have re ponded if were going', 'sooo sad', 'bullying me', ...,
       'yay good for both of you', 'but it wa worth it',
       'all thi flirting going on the atg mile yay hug'], dtype='<U133')

# Label Encoding

The data is categorical so , we need to convert the sentiment labels  from Neutral , Positive & Negative to a float type so your model can understand , & we are going to acheive this by using to_categorical method from keras

In [20]:
import tensorflow as tf
import keras

In [21]:
labels = np.array(train['sentiment'])
y = []
for i in range(len(labels)):
    if labels[i] == 'neutral':
        y.append(0)
    if labels[i] == 'positive':
        y.append(1)
    if labels[i] == 'negative':
        y.append(2)
        
y = np.array(y)
labels = tf.keras.utils.to_categorical(y , 3 , dtype='float32')
del y

In [22]:
len(labels)

27481

# Data Sequencing & Splitting

I am going to implement the keras tokenizer as well as its pad_sequences method to transform your data into 3D float data , otherwise your model is unable to trained it.

In [23]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras import backend as k
from keras.callbacks import ModelCheckpoint
max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequenecs = tokenizer.texts_to_sequences(data)
tweets = pad_sequences(sequenecs , maxlen=max_len)
print(tweets)

[[   0    0    0 ...   72  153   46]
 [   0    0    0 ...    0 1627  490]
 [   0    0    0 ...    0    0   10]
 ...
 [   0    0    0 ...  390   11    5]
 [   0    0    0 ...   20  568    3]
 [   0    0    0 ...  437  204  464]]


In [24]:
print(labels)

[[1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [25]:
#splitting the data
from sklearn.model_selection import train_test_split

X_train , X_test , Y_train , Y_test = train_test_split(tweets , labels , random_state = 0)

print(len(X_train) ,len(X_test) , len(Y_train) , len(Y_test))

20610 6871 20610 6871


# Model Building

In [28]:
LSTM_model = Sequential()
LSTM_model.add(layers.Embedding(max_words , 20))
LSTM_model.add(layers.LSTM(15 , dropout=0.5))
LSTM_model.add(layers.Dense(3,activation='softmax'))

LSTM_model.compile(optimizer='rmsprop',loss='categorical_crossentropy' , metrics=['accuracy'])

checkpoint = ModelCheckpoint("best_LSTM_model.hdf5",monitor='val_accuracy',verbose=1,save_best_only=True,mode='auto',period=1,save_weights_only=False)
history = LSTM_model.fit(X_train , Y_train , epochs=70 , validation_data=(X_test , Y_test) , callbacks=[checkpoint])

Epoch 1/70
645/645 [==============================] - ETA: 0s - loss: 0.7829 - accuracy: 0.6744
Epoch 1: val_accuracy improved from -inf to 0.74240, saving model to best_LSTM_model.hdf5
645/645 [==============================] - 74s 108ms/step - loss: 0.7829 - accuracy: 0.6744 - val_loss: 0.6263 - val_accuracy: 0.7424
Epoch 2/70
  1/645 [..............................] - ETA: 1:08 - loss: 0.7749 - accuracy: 0.6875

C:\Users\Acer\anaconda3\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


645/645 [==============================] - ETA: 0s - loss: 0.5665 - accuracy: 0.7752
Epoch 2: val_accuracy improved from 0.74240 to 0.77645, saving model to best_LSTM_model.hdf5
645/645 [==============================] - 67s 104ms/step - loss: 0.5665 - accuracy: 0.7752 - val_loss: 0.5476 - val_accuracy: 0.7765
Epoch 3/70
645/645 [==============================] - ETA: 0s - loss: 0.4998 - accuracy: 0.8073
Epoch 3: val_accuracy improved from 0.77645 to 0.80410, saving model to best_LSTM_model.hdf5
645/645 [==============================] - 66s 102ms/step - loss: 0.4998 - accuracy: 0.8073 - val_loss: 0.4972 - val_accuracy: 0.8041
Epoch 4/70
645/645 [==============================] - ETA: 0s - loss: 0.4731 - accuracy: 0.8211
Epoch 4: val_accuracy improved from 0.80410 to 0.81022, saving model to best_LSTM_model.hdf5
645/645 [==============================] - 66s 102ms/step - loss: 0.4731 - accuracy: 0.8211 - val_loss: 0.4858 - val_accuracy: 0.8102
Epoch 5/70
645/645 [======================

In [30]:
best_model = keras.models.load_model("best_LSTM_model.hdf5")
test_loss , test_acc = best_model.evaluate(X_test , Y_test , verbose =2)
print("Model Accuracy:" , test_acc)

215/215 - 8s - loss: 0.4438 - accuracy: 0.8424 - 8s/epoch - 35ms/step
Model Accuracy: 0.8423810005187988


The accuracy score of the model is 84% which is very good but it is not a good practice to find out how good your model is.

In [31]:
sentiment = ['Neutral' , 'Positive' , 'Negative']


In [47]:
## Model Testing

sequence = tokenizer.texts_to_sequences(['sunny day'])
test = pad_sequences(sequence,maxlen=max_len)
print(sentiment[np.around(best_model.predict(test),decimals=0).argmax(axis=1)[0]])

1/1 [==============================] - 0s 49ms/step
Neutral
